<a href="https://colab.research.google.com/github/AndresPerelmuter/E-commerce-Product-Images/blob/main/E_commerce_Product_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import shutil
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers, models


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
base_dir = "/content/drive/MyDrive/E-commerce Product Images/data"  # Reemplaza con la ruta de tu carpeta 'data'

In [4]:
# Inspección de la estructura de archivos
def explore_directory(path):
    for root, dirs, files in os.walk(path):
        print(f"Directorio: {root}")
        if dirs:
            print(f"  Subdirectorios: {dirs}")
        if files:
            print(f"  Archivos: {files[:5]}...")

# Explorar la carpeta base
explore_directory(base_dir)

Directorio: /content/drive/MyDrive/E-commerce Product Images/data
  Subdirectorios: ['Footwear', 'Apparel', 'Imagenes']
  Archivos: ['fashion.csv', '.DS_Store']...
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear
  Subdirectorios: ['Women', 'Men']
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear/Women
  Subdirectorios: ['Images']
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear/Women/Images
  Subdirectorios: ['images_with_product_ids']
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear/Women/Images/images_with_product_ids
  Archivos: ['13042.jpg', '12902.jpg', '11511.jpg', '10292.jpg', '11497.jpg']...
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear/Men
  Subdirectorios: ['Images']
Directorio: /content/drive/MyDrive/E-commerce Product Images/data/Footwear/Men/Images
  Subdirectorios: ['images_with_product_ids']
Directorio: /content/drive/MyDrive/E-commerce Pro

In [5]:
csv_path = os.path.join(base_dir, "fashion.csv")

data = pd.read_csv(csv_path)

In [6]:
data

,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL
0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...
1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...
2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,40143.jpg,http://assets.myntassets.com/v1/images/style/p...
3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,23623.jpg,http://assets.myntassets.com/v1/images/style/p...
4,47154,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris,47154.jpg,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...
2901,51755,Women,Footwear,Shoes,Casual Shoes,Black,Casual,Catwalk Women Black Shoes,51755.jpg,http://assets.myntassets.com/v1/images/style/p...
2902,47630,Women,Footwear,Shoes,Flats,Blue,Casual,Carlton London Women Blue Shoes,47630.jpg,http://assets.myntassets.com/v1/images/style/p...
2903,32836,Women,Footwear,Shoes,Flats,Pink,Casual,Grendha Women Flori Pink Sandals,32836.jpg,http://assets.myntassets.com/v1/images/style/p...
2904,35821,Women,Footwear,Shoes,Heels,Black,Casual,Enroute Women Black Heels,35821.jpg,http://assets.myntassets.com/v1/images/style/p...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ProductId     2906 non-null   int64 
 1   Gender        2906 non-null   object
 2   Category      2906 non-null   object
 3   SubCategory   2906 non-null   object
 4   ProductType   2906 non-null   object
 5   Colour        2906 non-null   object
 6   Usage         2906 non-null   object
 7   ProductTitle  2906 non-null   object
 8   Image         2906 non-null   object
 9   ImageURL      2906 non-null   object
dtypes: int64(1), object(9)
memory usage: 227.2+ KB


In [8]:
# Función para construir la ruta completa de la imagen
def construct_image_path(row):
    category = row["Category"]
    gender = row["Gender"]
    image_name = row["Image"]

    return os.path.join(base_dir, category, gender, "Images", "images_with_product_ids", image_name)

data["image_path"] = data.apply(construct_image_path, axis=1)

print(data.head())

   ProductId Gender Category SubCategory ProductType Colour   Usage  \
0      42419  Girls  Apparel     Topwear        Tops  White  Casual   
1      34009  Girls  Apparel     Topwear        Tops  Black  Casual   
2      40143  Girls  Apparel     Topwear        Tops   Blue  Casual   
3      23623  Girls  Apparel     Topwear        Tops   Pink  Casual   
4      47154  Girls  Apparel  Bottomwear      Capris  Black  Casual   

                                  ProductTitle      Image  \
0           Gini and Jony Girls Knit White Top  42419.jpg   
1                Gini and Jony Girls Black Top  34009.jpg   
2  Gini and Jony Girls Pretty Blossom Blue Top  40143.jpg   
3   Doodle Kids Girls Pink I love Shopping Top  23623.jpg   
4             Gini and Jony Girls Black Capris  47154.jpg   

                                            ImageURL  \
0  http://assets.myntassets.com/v1/images/style/p...   
1  http://assets.myntassets.com/v1/images/style/p...   
2  http://assets.myntassets.com/v1/ima

In [9]:
# Verificar si las imágenes existen
data["image_exists"] = data["image_path"].apply(os.path.exists)

# Imprimir las imágenes que faltan
missing_images = data[~data["image_exists"]]
if not missing_images.empty:
    print(f"Faltan {len(missing_images)} imágenes:")
    print(missing_images)

# Filtrar solo las imágenes existentes
data = data[data["image_exists"]]
print(f"Imágenes válidas: {len(data)}")


Imágenes válidas: 2906


In [10]:
# Preparar el dataset de TensorFlow
def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])  # Redimensionar a 128x128 píxeles
    img = img / 255.0  # Normalizar entre 0 y 1
    return img, label

# Convertir etiquetas categóricas a numéricas
data["gender_label"] = data["Gender"].astype("category").cat.codes

# Crear el dataset
image_paths = data["image_path"].values
labels = data["gender_label"].values
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(lambda x, y: load_image(x, y))
dataset = dataset.batch(32).shuffle(buffer_size=1000)


In [11]:
# Variables independientes (rutas de las imágenes) y dependientes (etiquetas)
image_paths = data["image_path"].values
labels = data["Gender"].astype("category").cat.codes.values

# División del dataset
x_train_paths, x_test_paths, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_paths, y_train))
train_dataset = train_dataset.map(load_image).batch(32).shuffle(buffer_size=1000)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test_paths, y_test))
test_dataset = test_dataset.map(load_image).batch(32)


In [13]:
# Definir el modelo CNN con Dropout
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(4, activation='softmax')
])


# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,
    batch_size=32,
    callbacks=[lr_scheduler]
)



Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 614s 3s/step - accuracy: 0.3095 - loss: 1.9165 - val_accuracy: 0.7113 - val_loss: 0.6382 - learning_rate: 0.0010
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7524 - loss: 0.5714 - val_accuracy: 0.8505 - val_loss: 0.4313 - learning_rate: 0.0010
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.8185 - loss: 0.4742 - val_accuracy: 0.7835 - val_loss: 0.5223 - learning_rate: 0.0010
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.8025 - loss: 0.4682 - val_accuracy: 0.8471 - val_loss: 0.4115 - learning_rate: 0.0010
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.8433 - loss: 0.3893 - val_accuracy: 0.8694 - val_loss: 0.3509 - learning_rate: 0.0010
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.8653 - loss: 0.3400 - val_accuracy: 0.8746 - val_loss: 0.3257 - learning_rate: 0.0010
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.8624 - loss: 0.3518 - val_acc

In [15]:
# Evaluar el modelo en los datos de prueba
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Pérdida en el conjunto de prueba: {test_loss}")
print(f"Precisión en el conjunto de prueba: {test_accuracy}")


19/19 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8998 - loss: 0.3326
Pérdida en el conjunto de prueba: 0.3538062572479248
Precisión en el conjunto de prueba: 0.8848797082901001


In [16]:
# Guardar el modelo entrenado
model.save("model_cnn.h5")
